# Création d'un nouveau df unique

Objectif: recréer un nouveau df unique.  
Préprocessing:  
    * suppression des nan de RainTomorrow et RainToday (règle le problème de Rainfall)  
    * remplacement des valeurs manquantes numériques par la moyenne globale  
    * Que faire des 3 WindGustDir?   
    * ajout des données des stations : coordonnées GPS, Code etat, grandeurs non mesurées  
    * création de trois colonnes: jour, mois, année  
 
    

In [18]:
import pandas as pd
import numpy as np

In [19]:
df = pd.read_csv('../data/raw/weatherAUS.csv')
df.head()

,Date,Location,MinTemp,MaxTemp,Rainfall,Evaporation,Sunshine,WindGustDir,WindGustSpeed,WindDir9am,...,Humidity9am,Humidity3pm,Pressure9am,Pressure3pm,Cloud9am,Cloud3pm,Temp9am,Temp3pm,RainToday,RainTomorrow
0,2008-12-01,Albury,13.4,22.9,0.6,NaN,NaN,W,44.0,W,...,71.0,22.0,1007.7,1007.1,8.0,NaN,16.9,21.8,No,No
1,2008-12-02,Albury,7.4,25.1,0.0,NaN,NaN,WNW,44.0,NNW,...,44.0,25.0,1010.6,1007.8,NaN,NaN,17.2,24.3,No,No
2,2008-12-03,Albury,12.9,25.7,0.0,NaN,NaN,WSW,46.0,W,...,38.0,30.0,1007.6,1008.7,NaN,2.0,21.0,23.2,No,No
3,2008-12-04,Albury,9.2,28.0,0.0,NaN,NaN,NE,24.0,SE,...,45.0,16.0,1017.6,1012.8,NaN,NaN,18.1,26.5,No,No
4,2008-12-05,Albury,17.5,32.3,1.0,NaN,NaN,W,41.0,ENE,...,82.0,33.0,1010.8,1006.0,7.0,8.0,17.8,29.7,No,No


In [20]:
gps =  pd.read_excel('../data/external/station_gps_region.xlsx')
gps.head()

,Location,Latitude,Longitude,CodeRegion,NonMes
0,Albury,-36.080477,146.916280,NSW,Evaporation_Sunshine
1,BadgerysCreek,-33.881667,150.744163,NSW,Evaporation_Sunshine_Cloud9am_Cloud3pm
2,Cobar,-31.498333,145.834444,NSW,all
3,CoffsHarbour,-30.296241,153.113529,NSW,all
4,Moree,-29.461720,149.840715,NSW,all


In [21]:
####### Création des colonnes year, month, date ###################
# Mise au format datetime de la colonne 'Date':
df['Date'] = pd.to_datetime(df['Date'])

# Extraction d'informations depuis la variable date
df["Year"] = df['Date'].dt.year # Année:
df["Month"] = df['Date'].dt.month # Mois
df["Day"] = df['Date'].dt.day # Jour du mois

####### Gestion des Rain catégorielles: ############################

# DropNa RainToday et Tomorrow:
df.dropna(subset = ['RainToday', 'RainTomorrow'], inplace = True) # On drop ici pour réaffecter le type de la colonne à entier.

# Numérisation de RainToday et Tomorrow:
df['RainToday'].replace(to_replace = ['Yes', 'No'], value = [1, 0], inplace = True)
df['RainToday'] = df['RainToday'].astype(int)

df['RainTomorrow'].replace(to_replace = ['Yes', 'No'], value = [1, 0], inplace = True)
df['RainTomorrow'] = df['RainTomorrow'].astype(int)

# Encodage des noms des stations:
df['LocationNum'] = df['Location']  # création d'une copie de la colonne 'Location'
df['LocationNum'].replace(to_replace = df['Location'].unique(), value =np.arange(len(df['Location'].unique())), inplace = True ) #remplacement par le code numérique.

############## Numérisation WindGustDir, WindDir9am, WindDir3pm #####

# Création d'un dictionnaire pour convertir, dans « WindGustDir », les directions du vent en radians
WindGustDir_directions_to_radians_dictionary = {
    "N": np.pi / 2,
    "E": 0.0,
    "S": 3 * np.pi,
    "W": np.pi,
    "NE": np.pi / 4,
    "SE": 7 * np.pi / 4,
    "SW": 5 * np.pi / 4,
    "NW": 3 * np.pi / 4,
    "NNE": 0.375 * np.pi,
    "ENE": 0.125 * np.pi,
    "ESE": 1.875 * np.pi,
    "SSE": 1.625 * np.pi,
    "SSW": 1.375 * np.pi,
    "WSW": 1.125 * np.pi,
    "WNW": 0.875 * np.pi,
    "NNW": 0.625 * np.pi
}

# Création des trois colonnes copies numériques:
df['WindGustDirNum'] = df['WindGustDir']
df['WindDir9amNum'] = df['WindDir9am']
df['WindDir3pmNum'] = df['WindDir3pm']

# Remplacement des catégories par les valeurs:
df['WindGustDirNum'].replace(to_replace = WindGustDir_directions_to_radians_dictionary, inplace = True)
df['WindDir9amNum'].replace(to_replace = WindGustDir_directions_to_radians_dictionary, inplace = True)
df['WindDir3pmNum'].replace(to_replace = WindGustDir_directions_to_radians_dictionary, inplace = True)

########## Ajout des données GPS #####################
# Fusion de df et gps pour ajouter les 4 colonnes de gps à df, à savoir: Latitude, Longitude, CodeRegion, NonMes
df = df.merge(right = gps, on = 'Location', how = 'inner')
df.head()

# Encodage des noms des régions:

df['CodeRegionNum'] = df['CodeRegion']  # création d'une copie de la colonne 'Location'
df['CodeRegionNum'].replace(to_replace = df['CodeRegion'].unique(), value =np.arange(len(df['CodeRegion'].unique())), inplace = True ) #remplacement par le code numérique.

# Encodage des noms des valeurs non mesurées:

df['NonMesNum'] = df['NonMes']  # création d'une copie de la colonne 'Location'
df['NonMesNum'].replace(to_replace = df['NonMes'].unique(), value =np.arange(len(df['NonMes'].unique())), inplace = True ) #remplacement par le code numérique.



In [26]:
############### Gestion des nan ##################
# Sélection des variables à remplacer par la moyenne
mean_var = ['MinTemp', 'MaxTemp', 'WindGustSpeed', 'WindSpeed9am', 'WindSpeed3pm', 
            'Humidity9am', 'Humidity3pm', 'Pressure9am', 'Pressure3pm', 'Cloud9am', 'Cloud3pm', 
            'Temp9am', 'Temp3pm', 'Evaporation', 'Sunshine',
            'WindGustDirNum', 'WindDir9amNum', 'WindDir3pmNum']
left_over = ['Rainfall']

#Sélection des variables catégorielles à remplacer par le mode:
mode_var =['WindGustDir', 'WindDir9am', 'WindDir3pm'] # sélection des variables catégorielles à remplacer

# Remplacement des nan par la moyenne ou le mode le plus fréquent:

for station in df['Location'].unique():
    dict_mean_values = {} # Initialisation dictionnaire pour les valeurs moyennes des variables numériques: clé = mesure, valeur = moyenne de la mesure sur tout le tableau
    dict_mode_values = {} # Initialisation dictionnaire pour les valeurs les plus fréquentes des variables catégorielles: clé = mesure, valeur = mode[0] de la mesure sur tout le tableau
    
    # La ligne du dessous permettait de remplacer les nan par la moyenne par station. Elle est inutile maintenant car on remplace brutalement les nan par la moyenne sur le tableau. A garder pour une prochaine update?
    # data = df.loc[df['Location'] == station].copy() # je n'ai pas trouvé d'autre solutions que celle ci, qui me parait un peu lourde niveau mémoire, puisqu'elle nécéssite de créer une copie pour chaque station. Mais je m'arête là car ça marche.
    
    for var in mean_var:   #Création du dictionnaire pour les valeurs moyennes des variables numériques
        if var == 'Cloud9am' or var == 'Cloud3pm':             # Test spécifique sur les Clouds pour arrondir la moyenne
            dict_mean_values[var] = round(df[var].mean(), 0)
        else:
            dict_mean_values[var] = df[var].mean()
    
    for var in mode_var:   #Création du dictionnaire pour les valeurs les plus fréquentes des variables catégorielles
        dict_mode_values[var] = df[var].mode()[0]
    
    df.fillna(value = dict_mean_values, inplace = True)
    df.fillna(value = dict_mode_values, inplace = True)
    #df.fillna(data, inplace = True) inutile ici: servait lors du remplacement des nan par la moyenne de chaque station.

In [27]:
# Vérification du taux de nan:
round(df.isna().sum()/df.shape[0] *100, 2)
# Les grandeurs sélectionnées sont à zéro: tout a bien fonctionné.

Date              0.0
Location          0.0
MinTemp           0.0
MaxTemp           0.0
Rainfall          0.0
Evaporation       0.0
Sunshine          0.0
WindGustDir       0.0
WindGustSpeed     0.0
WindDir9am        0.0
WindDir3pm        0.0
WindSpeed9am      0.0
WindSpeed3pm      0.0
Humidity9am       0.0
Humidity3pm       0.0
Pressure9am       0.0
Pressure3pm       0.0
Cloud9am          0.0
Cloud3pm          0.0
Temp9am           0.0
Temp3pm           0.0
RainToday         0.0
RainTomorrow      0.0
Year              0.0
Month             0.0
Day               0.0
LocationNum       0.0
WindGustDirNum    0.0
WindDir9amNum     0.0
WindDir3pmNum     0.0
Latitude          0.0
Longitude         0.0
CodeRegion        0.0
NonMes            0.0
CodeRegionNum     0.0
NonMesNum         0.0
dtype: float64

In [31]:
## Drop des grandeurs catégorielles:
cat_to_drop = ['Location', 'WindGustDir', 'WindDir9am', 'WindDir3pm', 'CodeRegion', 'NonMes']
df.drop(columns = cat_to_drop, inplace = True)

# Création du csv final prêt pour la modélisation
df.to_csv('../data/processed/model_weatherAUS.csv')